<a href="https://colab.research.google.com/github/Kuro2509/IA_ASCS/blob/main/ExamenASCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas nltk scikit-learn tensorflow tensorflowjs


In [ ]:
!pip install tensorflow==2.17.0 tensorflow-decision-forests==1.10.0 tf-keras==2.17.0

In [ ]:
!pip list | grep tensorflow

tensorflow                         2.17.0
tensorflow-datasets                4.9.6
tensorflow_decision_forests        1.10.0
tensorflow-hub                     0.16.1
tensorflow-io-gcs-filesystem       0.37.1
tensorflow-metadata                1.16.1
tensorflow-probability             0.24.0
tensorflowjs                       4.22.0


In [ ]:
!mkdir spam

mkdir: cannot create directory ‘spam’: File exists


In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import tensorflowjs as tfjs

# URL directa al archivo CSV en formato "raw" en GitHub
url = "https://raw.githubusercontent.com/Kuro2509/IA_ASCS/main/SPAM.csv"
df = pd.read_csv(url, sep=',', header=None, names=['label', 'message'], encoding='latin1')

# Preprocesamiento de texto
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

df['message'] = df['message'].fillna('').astype(str)
df['message'] = df['message'].str.lower()
df['message'] = df['message'].str.replace(r'\d+', '', regex=True)
df['message'] = df['message'].str.translate(str.maketrans('', '', string.punctuation))
df['message'] = df['message'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))
df['message'] = df['message'].apply(lambda x: ' '.join(stemmer.stem(word) for word in x.split()))
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# División de datos
X_train, X_valid, y_train, y_valid = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# Tokenización y Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_valid_sequences = tokenizer.texts_to_sequences(X_valid)

max_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_valid_padded = pad_sequences(X_valid_sequences, maxlen=max_length, padding='post', truncating='post')

# Definir el modelo
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    LSTM(units=64, return_sequences=False),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento
epochs = 5
history = model.fit(X_train_padded, y_train, epochs=epochs, validation_data=(X_valid_padded, y_valid))

# Guardar el modelo en TensorFlow.js
!mkdir -p spam
tfjs.converters.save_keras_model(model, "spam")
print("Modelo guardado exitosamente en la carpeta 'spam' como model.json y archivos de pesos.")


Epoch 1/5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


128/128 ━━━━━━━━━━━━━━━━━━━━ 11s 56ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/5
128/128 ━━━━━━━━━━━━━━━━━━━━ 11s 63ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 3/5
128/128 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 4/5
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 5/5
128/128 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan


failed to lookup keras version from the file,
    this is likely a weight only file
Modelo guardado exitosamente en la carpeta 'spam' como model.json y archivos de pesos.


In [ ]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ (None, 100, 50)             │              50 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 64)                  │          29,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 88,667 (346.36 KB)

 Trainable params: 29,555 (115.45 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 59,112 (230.91 KB)